# Tracking Performance Evaluation

In [ ]:
# import py-motmetrics
!pip install motmetrics

  ## MOT Metrics Calculation

* py-motmetrics 패키지를 활용하여 추적 성능을 측정 ([MOT16](https://arxiv.org/abs/1603.00831) Challenge 기반)
* metrics 계산 함수는 [py-mottmotrics의 예시 함수](https://github.com/cheind/py-motmetrics?tab=readme-ov-file#for-custom-dataset) 코드를 기반으로 함
* 해당 ipynb파일에서는 ByteTrack 수정 전후의 공에 대한 추적 성능을 비교하기 위해 ```only_ball=True``` argument를 사용

In [1]:
# define a custom data evaluation function
def motMetricsEnhancedCalculator(gtSource, tSource, only_ball=False):
    """
    ground truth annotation과 tracking prediction annnotion을 통해 MOT metrics를 계산하고 결과를 출력함
    
        :gtSource (string): ground truth annotation (.txt) 파일 경로
        :tSource (string): tracking prediction annotation (.txt) 파일 경로
        :only_ball (bool): True라면 tennis_ball 클래스(1)에 대해서만 metrics 계산,
                            False일 경우 선수(0)와 공(1) 모두에 대해 계산
    """
    # import required packages
    import motmetrics as mm
    import numpy as np

    # load ground truth
    gt = np.loadtxt(gtSource, delimiter=',')

    # load tracking output
    t = np.loadtxt(tSource, delimiter=',')
    
    
    # if you want to calculate metrics only with tennis_ball class(1)
    if only_ball:
        gt = gt[gt[:,7] == 1]
        t = t[t[:,7] == 1]

    # Create an accumulator that will be updated during each frame
    acc = mm.MOTAccumulator(auto_id=True)

    # Max frame number maybe different for gt and t files
    for frame in range(int(gt[:,0].max())):
        frame += 1 # detection and frame numbers begin at 1

        # select id, x, y, width, height for current frame
        # required format for distance calculation is X, Y, Width, Height \
        # We already have this format
        gt_dets = gt[gt[:,0]==frame,1:6] # select all detections in gt
        t_dets = t[t[:,0]==frame,1:6] # select all detections in t

        C = mm.distances.iou_matrix(gt_dets[:,1:], t_dets[:,1:], \
                                    max_iou=0.5) # format: gt, t

        # Call update once for per frame.
        # format: gt object ids, t object ids, distance
        acc.update(gt_dets[:,0].astype('int').tolist(), \
                  t_dets[:,0].astype('int').tolist(), C)

    mh = mm.metrics.create()

    summary = mh.compute(acc, metrics=['num_frames', 'idf1', 'idp', 'idr', 
                                     'recall', 'precision', 'num_objects', 
                                     'mostly_tracked', 'partially_tracked', 
                                     'mostly_lost', 'num_false_positives', 
                                     'num_misses', 'num_switches', 
                                     'num_fragmentations', 'mota', 'motp', 
                                       'num_detections'
                                    ], 
                      name='acc')

    strsummary = mm.io.render_summary(
      summary,
      #formatters={'mota' : '{:.2%}'.format},
      namemap={'idf1': 'IDF1', 'idp': 'IDP', 'idr': 'IDR', 'recall': 'Rcll', 
               'precision': 'Prcn', 'num_objects': 'GT', 
               'mostly_tracked' : 'MT', 'partially_tracked': 'PT', 
               'mostly_lost' : 'ML', 'num_false_positives': 'FP', 
               'num_misses': 'FN', 'num_switches' : 'IDsw', 
               'num_fragmentations' : 'FM', 'mota': 'MOTA', 'motp' : 'MOTP', 
               'num_detections' : 'DET'
              }
    )
    print(strsummary)

In [2]:
# Calculate the MOT metrics for modified ByteTrack

print('MOT metrics of <<HARD>> court :')
motMetricsEnhancedCalculator('evaluation/test_annotation/TENNIS-HARD-GT.txt', 
                             'evaluation/test_annotation/TENNIS-HARD-PRED.txt',
                              only_ball=True)

print('MOT metrics of <<GRASS>> court :')
motMetricsEnhancedCalculator('evaluation/test_annotation/TENNIS-GRASS-GT.txt', 
                             'evaluation/test_annotation/TENNIS-GRASS-PRED.txt',
                              only_ball=True)

print('MOT metrics of <<CLAY>> court :')
motMetricsEnhancedCalculator('evaluation/test_annotation/TENNIS-CLAY-GT.txt', 
                             'evaluation/test_annotation/TENNIS-CLAY-PRED.txt',
                              only_ball=True)

MOT metrics of <<HARD>> court :
     num_frames      IDF1       IDP       IDR      Rcll      Prcn   GT  MT  PT  ML  FP  FN  IDsw  FM      MOTA      MOTP  DET
acc         359  0.286149  0.294671  0.278107  0.908284  0.962382  338   1   0   0  12  31     3  12  0.863905  0.115443  307
MOT metrics of <<GRASS>> court :
     num_frames      IDF1       IDP       IDR      Rcll      Prcn   GT  MT  PT  ML  FP  FN  IDsw  FM      MOTA      MOTP  DET
acc         349  0.417549  0.414414  0.420732  0.878049  0.864865  328   1   0   0  45  40     2  16  0.734756  0.129448  288
MOT metrics of <<CLAY>> court :
     num_frames      IDF1       IDP       IDR      Rcll      Prcn   GT  MT  PT  ML  FP  FN  IDsw  FM     MOTA      MOTP  DET
acc         442  0.212766  0.218447  0.207373  0.854839  0.900485  434   1   0   0  41  63     7  21  0.74424  0.117773  371


In [3]:
# Calculate the MOT metrics for default ByteTrack (baseline)
print('MOT metrics of <<HARD>> court (default ByteTrack) :')
motMetricsEnhancedCalculator('evaluation/test_annotation/TENNIS-HARD-GT.txt', 
                             'evaluation/test_annotation/TENNIS-HARD-PRED-BYTETRACK.txt',
                              only_ball=True)

print('MOT metrics of <<GRASS>> court (default ByteTrack) :')
motMetricsEnhancedCalculator('evaluation/test_annotation/TENNIS-GRASS-GT.txt', 
                             'evaluation/test_annotation/TENNIS-GRASS-PRED-BYTETRACK.txt',
                              only_ball=True)

print('MOT metrics of <<CLAY>> court (default ByteTrack) :')
motMetricsEnhancedCalculator('evaluation/test_annotation/TENNIS-CLAY-GT.txt', 
                             'evaluation/test_annotation/TENNIS-CLAY-PRED-BYTETRACK.txt',
                              only_ball=True)

MOT metrics of <<HARD>> court (default ByteTrack) :
     num_frames      IDF1       IDP       IDR      Rcll      Prcn   GT  MT  PT  ML  FP   FN  IDsw  FM      MOTA      MOTP  DET
acc         359  0.257391  0.312236  0.218935  0.683432  0.974684  338   0   1   0   6  107     8  12  0.642012  0.101981  231
MOT metrics of <<GRASS>> court (default ByteTrack) :
     num_frames      IDF1       IDP       IDR      Rcll     Prcn   GT  MT  PT  ML  FP  FN  IDsw  FM      MOTA      MOTP  DET
acc         349  0.141509  0.146104  0.137195  0.807927  0.86039  328   1   0   0  43  63    15  22  0.631098  0.123922  265
MOT metrics of <<CLAY>> court (default ByteTrack) :
     num_frames      IDF1       IDP       IDR      Rcll      Prcn   GT  MT  PT  ML  FP   FN  IDsw  FM      MOTA      MOTP  DET
acc         442  0.102696  0.115942  0.092166  0.711982  0.895652  434   0   1   0  36  125    22  27  0.578341  0.105572  309


## Inference Time Calculation

* 별도로 저장한 트래킹 로그(/evaluation/tracking_logs.py)에서 평균 추론시간을 계산

In [4]:
def average_processing_times(log_text):
    """
    트래킹 로그로부터 평균 처리 시간(preprocess, inference, postprocess time)을 계산 
    
        :log_text (string): 한 영상 입력에 대한 YOLO_model.track메소드 실행 결과 텍스트
        
        :average_preprocess_time (float):  한 프레임당 평균 preprocess time (ms)
        :average_inference_time (float): 한 프레임당 평균 inference time (ms)
        :average_postprocess_time (float): 한 프레임당 평균 postprocess time (ms)
    """
    import re
    preprocess_times = []
    inference_times = []
    postprocess_times = []

    matches = re.finditer(r'Speed: (\d+\.\d+)ms preprocess, (\d+\.\d+)ms inference, (\d+\.\d+)ms postprocess', log_text)
    for match in matches:
        preprocess_time = float(match.group(1))
        inference_time = float(match.group(2))
        postprocess_time = float(match.group(3))

        preprocess_times.append(preprocess_time)
        inference_times.append(inference_time)
        postprocess_times.append(postprocess_time)

    average_preprocess_time = sum(preprocess_times) / len(preprocess_times) if preprocess_times else None
    average_inference_time = sum(inference_times) / len(inference_times) if inference_times else None
    average_postprocess_time = sum(postprocess_times) / len(postprocess_times) if postprocess_times else None

    return average_preprocess_time, average_inference_time, average_postprocess_time

In [5]:
# 함수 호출 및 출력
from evaluation.tracking_logs import hard_logs_text, grass_logs_text, clay_logs_text # tracking_logs.py에서 각 로그(string) 불러오기

logs = [hard_logs_text, grass_logs_text, clay_logs_text]
courts = ['HARD', 'GRASS', 'CLAY']

for court, log in zip(courts, logs):
    average_preprocess, average_inference, average_postprocess = average_processing_times(log)
    
    print('<<'+court+'>> court inference time: ')
    
    if average_preprocess is not None:
        print(f"Average Preprocess Time: {average_preprocess}ms")

    if average_inference is not None:
        print(f"Average Inference Time: {average_inference}ms")

    if average_postprocess is not None:
        print(f"Average Postprocess Time: {average_postprocess}ms")

    if all([average_preprocess, average_inference, average_postprocess]):
        total_average_time = average_preprocess + average_inference + average_postprocess
        print(f"Total Average Time: {total_average_time}ms")
    else:
        print("Some processing time data not found in the logs.")
    print()

<<HARD>> court inference time: 
Average Preprocess Time: 1.546239554317549ms
Average Inference Time: 8.81699164345404ms
Average Postprocess Time: 1.2997214484679664ms
Total Average Time: 11.662952646239555ms

<<GRASS>> court inference time: 
Average Preprocess Time: 1.5650429799426935ms
Average Inference Time: 8.959025787965617ms
Average Postprocess Time: 1.3200573065902579ms
Total Average Time: 11.844126074498568ms

<<CLAY>> court inference time: 
Average Preprocess Time: 1.4076923076923078ms
Average Inference Time: 7.866742081447963ms
Average Postprocess Time: 1.1778280542986423ms
Total Average Time: 10.452262443438913ms

